<a href="https://colab.research.google.com/github/swarathesh/Spark_ML/blob/master/Linear_regression_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Given features estimate the required crew 

In [25]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [5]:
spark = SparkSession.builder.appName('Cruise').getOrCreate()

In [6]:
df = spark.read.csv('cruise_ship_info.csv',header=True,inferSchema=True)

In [7]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [8]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [19]:
df.groupBy('Cruise_line').count().orderBy('count').show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|           Orient|    1|
|           Disney|    2|
|          Azamara|    2|
|          Crystal|    2|
|         Windstar|    3|
|          Oceania|    3|
|         Seabourn|    3|
|           Cunard|    3|
|        Silversea|    4|
|Regent_Seven_Seas|    5|
|             Star|    6|
|              P&O|    6|
|              MSC|    8|
|        Celebrity|   10|
|            Costa|   11|
|        Norwegian|   13|
| Holland_American|   14|
|         Princess|   17|
|         Carnival|   22|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [21]:
indexer  =  StringIndexer(inputCol='Cruise_line',outputCol='Cruise_line_index')

In [23]:
indexed = indexer.fit(df).transform(df)

In [26]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [27]:
vecAssembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'Cruise_line_index'],outputCol='features')

In [29]:
data = vecAssembler.transform(indexed)

In [34]:
data = data.select(['features','crew'])

In [35]:
train, test = data.randomSplit([0.8,0.2])

In [37]:
model = LinearRegression(labelCol='crew')

In [40]:
model_lr = model.fit(train)

In [41]:
pred = model_lr.evaluate(test)

In [43]:
pred.rootMeanSquaredError

0.633841541151926

In [44]:
pred.r2

0.970337379470097